# Liczenie Sredniego Doochodu brutto dla jednostek samorzadu terytorialnego


In [1]:
%pip install -e "./Pitmodule"
from Pitmodule.pitframe import *
from Pitmodule.population import PopulationFrame,default_tables,age_cond
import pandas as pd

Obtaining file:///C:/Users/adomoslaws001/nypd-finalassignment/Pitmodule
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: Pitmodule
    Found existing installation: Pitmodule 1.0
    Uninstalling Pitmodule-1.0:
      Successfully uninstalled Pitmodule-1.0
  Running setup.py develop for Pitmodule
Note: you may need to restart the kernel to use updated packages.


In [2]:
pit2020 = PitFrame(year=2020)
populacje2020 = PopulationFrame()

__Legenda__:

 abitralnie uznaje ze udzialy w subewmcjach wynosza odpowiednio: 39.25 %, 10.25 % , 1.6 % (wedlug ustawy o PIT'cie)
 wszysycy sa wedlug progu podatkowego  17% 


- dochod - dochod butto (z podatkiem) dla danej jst 
 $$ dochod = \frac{naleznosci}{0.17 * p_i} $$ 
 gdzie $p_i$ - udzial w subewncjach (0.3925, 0.1025, 0.016),

- dochod_per_capita
$$ \frac{dochod}{populacja_{jst}} $$


In [3]:
# Sredni dochod dla kazdej jst
# Gminy
gminy_pit = pit2020.data["Gminy"]
miasta_gminy_pit= pit2020.data['Miasta_Gminy']
avg_gminy =  populacje2020.count_avg_gminy(gminy_pit,miasta_gminy_pit)
avg_gminy

,jst_x,ludnosc,naleznosci,id,short id,dochod,dochod_per_capita
0,Bolesławiec,29284,40456699.0,0201011,0201,6.063200e+08,20704.820210
1,Bolesławiec,10387,13789712.0,0201022,0201,2.066648e+08,19896.490638
2,Gromadka,3841,3573540.0,0201032,0201,5.355624e+07,13943.305986
3,Nowogrodziec,10631,9553744.0,0201043,0201,1.431809e+08,13468.241627
4,Osiecznica,5036,5844560.0,0201052,0201,8.759176e+07,17393.120971
...,...,...,...,...,...,...,...
61,Leszno,45906,85694927.0,NaN,3063,1.284300e+09,27976.738169
62,Poznań,403143,942116546.0,NaN,3064,1.411939e+10,35023.288809
63,Koszalin,80478,131423957.0,NaN,3261,1.969636e+09,24474.215606
64,Szczecin,300952,489794370.0,NaN,3262,7.340493e+09,24390.908248


In [4]:
# Powiaty
powiaty_pit = pit2020.data["Powiaty"]
miasta_powiaty_pit= pit2020.data['Miasta_Powiaty']
avg_powiaty = populacje2020.count_avg_powiaty(powiaty_pit,miasta_powiaty_pit)
avg_powiaty

,jst_x,ludnosc,naleznosci,id,short id,dochod,dochod_per_capita
0,bolesławiecki,63524,21276533.0,0201,02,1.221035e+09,19221.631075
1,dzierżoniowski,72995,21328692.0,0202,02,1.224028e+09,16768.658611
2,głogowski,63068,28916309.0,0203,02,1.659473e+09,26312.433238
3,górowski,23769,5682116.0,0204,02,3.260899e+08,13719.124527
4,jaworski,35668,10450783.0,0205,02,5.997580e+08,16815.016020
...,...,...,...,...,...,...,...
61,m.Leszno,44704,23018175.0,3063,30,1.320986e+09,29549.607480
62,m.Poznań,388485,253058193.0,3064,30,1.452271e+10,37382.932031
63,m.Koszalin,77659,35301268.0,3261,32,2.025898e+09,26087.095290
64,m.Szczecin,290251,131561724.0,3262,32,7.550171e+09,26012.556974


In [5]:
# Wojewodztwa
wojew_pit = pit2020.data["Wojewodztwa"]
populacje2020.count_avg_wojewodztwa(wojew_pit)

,jst_x,ludnosc,naleznosci,id,dochod,dochod_per_capita
0,Dolnośląskie,2142808,143787146.0,02,5.286292e+10,24669.929048
1,Kujawsko-pomorskie,1488663,77938448.0,04,2.865384e+10,19248.037451
2,Lubelskie,1515929,65228210.0,06,2.398096e+10,15819.315785
3,Lubuskie,729556,38895731.0,08,1.429990e+10,19600.827219
4,Łódzkie,1801114,106927491.0,10,3.931158e+10,21826.257291
5,Małopolskie,2423229,154769447.0,12,5.690053e+10,23481.285502
6,Mazowieckie,3894033,380227237.0,14,1.397894e+11,35898.366903
7,Opolskie,731815,36183042.0,16,1.330259e+10,18177.529800
8,Podkarpackie,1514961,65682244.0,18,2.414788e+10,15939.607570
9,Podlaskie,851337,40759371.0,20,1.498506e+10,17601.799132


# Liczenia Wariancji docodow brutto dla wojewodztw/powiatow

Wariancja: $$\frac{\sum_{i=1}^{N} (X - \hat{X})^2}{n}$$

In [6]:
# Dla powiatow 'part var' oznacza wariancje dla danego powiatu
populacje2020.var_powiaty()

,short id,jst_x_pow,part_var
0,0201,bolesławiecki,1.453646e+07
1,0202,dzierżoniowski,9.277399e+06
2,0203,głogowski,7.051909e+07
3,0204,górowski,7.870247e+06
4,0205,jaworski,8.103294e+06
...,...,...,...
374,3217,wałecki,1.232217e+07
375,3218,łobeski,1.181947e+07
376,3261,m.Koszalin,2.601381e+06
377,3262,m.Szczecin,2.629745e+06


In [10]:
#wojewodztwo
populacje2020.variance_wojewodztwa()

,short id,jst_x_pow,part_var
0,02,Dolnośląskie,4.815550e+07
1,04,Kujawsko-pomorskie,2.026390e+07
2,06,Lubelskie,1.609723e+07
3,08,Lubuskie,1.271914e+07
4,10,Łódzkie,2.073539e+07
5,12,Małopolskie,5.448233e+07
6,14,Mazowieckie,2.900523e+08
7,16,Opolskie,1.507077e+07
8,18,Podkarpackie,1.235393e+07
9,20,Podlaskie,2.057349e+07
